In [ ]:
import kwant
import functools as ft
import matplotlib.pyplot as plt
import numpy as np
import adaptive
adaptive.notebook_extension()

import scipy.constants
import cmath

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import sns_system
import kwant

import distributed_sns as ds
import holoviews as hv

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin,
    abs=abs
   )

params_raw= dict(Ez=constants['mu_B']*26,
                 mu_n = 20,
                 mu_s = 20,
                 alpha = 20,
                 theta=np.pi/4,
                k_cutoff = 0.0915)

params = dict(**constants,
              **params_raw)

a= .1
W = 200

In [ ]:
def get_templates(with_SOy=False):
    ham_n = """(hbar^2 / (2*m_eff) * (k_x^2 + k_y^2) - mu_n) * sigma_0
    +  alpha*k_x*sigma_y -  alpha*k_y*sigma_x + Ez*(cos(theta)*sigma_x + sin(theta)*sigma_y)
    """
    if not with_SOy:
        ham_n = ham_n.replace('alpha*k_y', '0')
    
    ham_np = ham_n.replace('Ez', '0').replace('alpha', '0')
    print(ham_n)
    print(ham_np)
    
    template_np = kwant.continuum.discretize(
        ham_np, grid_spacing=a, coords=('x', 'y'))
    template_n = kwant.continuum.discretize(
        ham_n, grid_spacing=a, coords=('x', 'y'))
    
    return (template_n, template_np)

In [ ]:
(template_n, template_np) = get_templates(with_SOy=True)
sigmaz = np.array([[1,0],[0,-1]])
sigmay = 1j*np.array([[0,-1j],[1j,0]])

# lead_down = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,-a)), conservation_law=sigmaz, time_reversal=sigmay)
# lead_up = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,a)), conservation_law=sigmaz, time_reversal=sigmay)
lead_down = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,-a)))
lead_up = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,a)))

junction = kwant.Builder(kwant.TranslationalSymmetry((a,0)))

lead_down.fill(template_np, lambda x: True, [0, 0])
lead_up.fill(template_np, lambda x: True, [0, 0])
junction.fill(template_n, lambda site: 0 <= site.pos[1] <= W, [0, 0])

junction = kwant.wraparound.wraparound(junction)
lead_down = kwant.wraparound.wraparound(lead_down, keep=1)
lead_up = kwant.wraparound.wraparound(lead_up, keep=1)

junction.attach_lead(lead_down)
junction.attach_lead(lead_up)

syst = junction.finalized()

In [ ]:
def energy_operator(syst, params):
    if params['k_x'] == 0:
        smat_plus = smat_min = kwant.smatrix(syst, params=params).data
    else:
        params['k_x'] *= -1
        smat_min = kwant.smatrix(syst, params=params).data
        params['k_x'] *= -1
        smat_plus = kwant.smatrix(syst, params=params).data

    phase = cmath.exp(1j * params['phi'] / 2)
    r = np.diag([phase, phase, phase.conjugate(), phase.conjugate()])
    smat_prod = r @ smat_min.conj() @ r.conj() @ smat_plus
    return smat_prod


def energies_over_delta(syst, params):
    """Same as energy_operator(), but returns the
    square-root of the eigenvalues"""
    from numpy.lib.scimath import sqrt
    operator = energy_operator(syst, params)
    es = -np.linalg.eigvals(operator)
    alpha = np.hstack((sqrt(es), -sqrt(es)))
    alpha = alpha[np.imag(alpha)<=0]

    return np.sort(np.real((1+alpha**2)/(2*alpha)))

In [ ]:
def abs_min_log_loss(xs, ys):
    from adaptive.learner.learner1D import default_loss
    ys = [np.log(np.abs(y).min()) for y in ys]
    return default_loss(xs, ys)

def f(kx):
    
    return energies_over_delta(syst, params=dict(params, k_x=kx*a, phi=np.pi))
#     return np.min(Es)
    Es = np.array(Es)
    return np.hstack([Es, -Es])
                               
learner = adaptive.Learner1D(f, [-0.102463344877, 0.102463344877], loss_per_interval=abs_min_log_loss)

In [ ]:
def abs_min_log_loss(xs, ys):
    from adaptive.learner.learner1D import default_loss
    ys = [np.log(np.abs(y).min()) for y in ys]
    return default_loss(xs, ys)

def f(kx):
    
    return energies_over_delta(syst, params=dict(params, k_x=kx*a, phi=np.pi))
                               
learner = adaptive.Learner1D(f, [-0.102463344877, 0.102463344877])#, loss_per_interval=abs_min_log_loss)

In [ ]:
def abs_min_log_loss(xs, ys):
    from adaptive.learner.learner1D import default_loss
    ys = [np.log(np.abs(y).min()) for y in ys]
    return default_loss(xs, ys)

def f(kx):
    
    return np.min(np.abs(energies_over_delta(syst, params=dict(params, k_x=kx*a, phi=np.pi))))
                               
learner = adaptive.Learner1D(f, [0, 0.102463344877])#, loss_per_interval=abs_min_log_loss)

In [ ]:
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
gaps = np.array(sorted(learner.data.items()))

ks_   = gaps[:,0]
gaps_ = np.minimum.accumulate(gaps[:,1])
gap_before_k = scipy.interpolate.interp1d(ks_, gaps_)

In [ ]:
hv.Curve((ks_, gaps_)) *hv.Curve((ks_, gap_before_k(ks_)))

In [ ]:
kf-params['m_eff']*params['alpha']/params['hbar']**2

In [ ]:
kf = np.sqrt(params['mu_n']*2*params['m_eff'])/params['hbar']
lambda_f = 2*np.pi/kf
print(kf, lambda_f)

In [ ]:
np.sqrt(zx**2 + W**2)-D

In [ ]:
kf = lambda alpha, mu: constants['m_eff']*alpha/constants['hbar']**2 - np.sqrt(-(constants['m_eff']*alpha/constants['hbar'])**2 + constants['m_eff']*2*mu)/constants['hbar']

In [ ]:
kf(20, 20)

In [ ]:
zx = 4
zy = 1
def cutoff(zx, zy, W, params):
    D = np.sqrt(zx**2 + (2*zy + W * np.sqrt(1 + (4*zy/zx)**2))**2)
    kf = np.sqrt(params['mu_n']*2*params['m_eff'])/params['hbar']
#     return np.sqrt(1 - W**2/D**2)*0.094
    return 0.0915

magnitude = np.linspace(140, 400)
cutoffs = [cutoff(m*zx, m*zy, W, params) for m in magnitude]
hv.Curve((magnitude, cutoffs))

In [ ]:
gap = [gap_before_k(cutoff) for cutoff in cutoffs]

In [ ]:
hv.Curve((magnitude, np.array(gap)))

In [ ]:
new_data = {}
folding = (np.pi/a)/(1/a)
cut_off = 0.22116096894035706
for k, v in learner.data.items():
    if np.abs(k)<cut_off:
        new_data[np.fmod(k, folding)] = v

x = np.sort(np.array(list(new_data.keys())))
y = np.vstack([new_data[x_] for x_ in x])

points = []
for x_, ys in zip(x,y):
    for y_ in ys:
        points.append((x_, y_))

In [ ]:
hv.Points(points)

In [ ]:
learner.plot()


In [ ]:
learner.plot()


## NS interface

In [ ]:
def get_templates_ns():
    ham_n = """(hbar^2 / (2*m_eff) * (k_x^2 + k_y^2) - mu_n) * sigma_z
    """
    ham_s = """(hbar^2 / (2*m_eff) * (k_x^2 + k_y^2) - mu_s) * sigma_z + Delta*(cos(phi)*sigma_x + sin(phi)*sigma_y)
"""
    print(ham_n)
    print(ham_s)
    
    template_s = kwant.continuum.discretize(
        ham_s, grid_spacing=a, coords=('x', 'y'))
    template_n = kwant.continuum.discretize(
        ham_n, grid_spacing=a, coords=('x', 'y'))
    
    return (template_n, template_s)

In [ ]:
(template_n, template_s) = get_templates_ns()

# lead_down = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,-a)), conservation_law=sigmaz, time_reversal=sigmay)
# lead_up = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,a)), conservation_law=sigmaz, time_reversal=sigmay)
lead_down = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,-a)))
lead_up = kwant.Builder(kwant.TranslationalSymmetry((a,0),(0,a)))

junction = kwant.Builder(kwant.TranslationalSymmetry((a,0)))

lead_up.fill(template_n, lambda x: True, [0, 0])
junction.fill(template_n, lambda site: site.pos[1]==0, [0, 0])
lead_down.fill(template_s, lambda x: True, [0, 0])

junction = kwant.wraparound.wraparound(junction)
lead_down = kwant.wraparound.wraparound(lead_down, keep=1)
lead_up = kwant.wraparound.wraparound(lead_up, keep=1)

junction.attach_lead(lead_down)
junction.attach_lead(lead_up)

syst_ns = junction.finalized()

In [ ]:
kwant.smatrix(syst_ns, 0, params=dict(params, k_x=0.06*a, phi=np.pi, theta=0, Ez=.15, alpha=20, Delta=1))

In [ ]:
def d(zy):
    W = 200
    zx = 4*zy
    zy = zy
    
    d1 = np.sqrt((zx**2)/4 + (2*zy-np.sqrt(2)*W)**2)
    d2 = W / np.sqrt(d1**2 - W**2) * d1
    
    w1 = np.sqrt(d1**2 - W**2)
    w2 = W
    return (d1+2*d2, (w1 + 2*W)/(d1+2*d2))

In [ ]:
d(140)

In [ ]:
hv.notebook_extension('bokeh')

In [ ]:
%%opts Curve [logy=False]
zy = np.linspace(80,400,4000)
hv.Curve((zy, d(zy))).redim(y='Dmax(nm)',x='zy(nm)')

In [ ]:
%opts?

In [ ]:
np.sqrt(2)*200*3